In [193]:
import pandas as pd

In [194]:
df = pd.read_csv('data/countypres_2000-2020.csv')

In [195]:
df.head()

,year,state,state_po,county_name,county_fips,office,candidate,party,candidatevotes,totalvotes,version,mode
0,2000,ALABAMA,AL,AUTAUGA,1001.0,US PRESIDENT,AL GORE,DEMOCRAT,4942,17208,20220315,TOTAL
1,2000,ALABAMA,AL,AUTAUGA,1001.0,US PRESIDENT,GEORGE W. BUSH,REPUBLICAN,11993,17208,20220315,TOTAL
2,2000,ALABAMA,AL,AUTAUGA,1001.0,US PRESIDENT,RALPH NADER,GREEN,160,17208,20220315,TOTAL
3,2000,ALABAMA,AL,AUTAUGA,1001.0,US PRESIDENT,OTHER,OTHER,113,17208,20220315,TOTAL
4,2000,ALABAMA,AL,BALDWIN,1003.0,US PRESIDENT,AL GORE,DEMOCRAT,13997,56480,20220315,TOTAL


In [196]:
df.isna().sum()

year               0
state              0
state_po           0
county_name        0
county_fips       52
office             0
candidate          0
party              0
candidatevotes     0
totalvotes         0
version            0
mode               0
dtype: int64

In [197]:
df = df.dropna()

In [198]:
df['county_fips'] = df['county_fips'].astype(int)
df['county_fips'] = df['county_fips'].astype(str)
df['county_fips'] = df['county_fips'].astype(str).apply(lambda x: x.zfill(5) if len(x) != 5 else x)

In [199]:
df.head()

,year,state,state_po,county_name,county_fips,office,candidate,party,candidatevotes,totalvotes,version,mode
0,2000,ALABAMA,AL,AUTAUGA,01001,US PRESIDENT,AL GORE,DEMOCRAT,4942,17208,20220315,TOTAL
1,2000,ALABAMA,AL,AUTAUGA,01001,US PRESIDENT,GEORGE W. BUSH,REPUBLICAN,11993,17208,20220315,TOTAL
2,2000,ALABAMA,AL,AUTAUGA,01001,US PRESIDENT,RALPH NADER,GREEN,160,17208,20220315,TOTAL
3,2000,ALABAMA,AL,AUTAUGA,01001,US PRESIDENT,OTHER,OTHER,113,17208,20220315,TOTAL
4,2000,ALABAMA,AL,BALDWIN,01003,US PRESIDENT,AL GORE,DEMOCRAT,13997,56480,20220315,TOTAL


In [200]:
district_fips = df[
    (df['state'] == 'ALASKA') &
    (df['county_name'].str.contains('district', case=False, na=False))
]['county_fips'].unique()


In [201]:
district_fips

array(['02001', '02002', '02003', '02004', '02005', '02006', '02007',
       '02008', '02009', '02010', '02011', '02012', '02013', '02014',
       '02015', '02016', '02017', '02018', '02019', '02020', '02021',
       '02022', '02023', '02024', '02025', '02026', '02027', '02028',
       '02029', '02030', '02031', '02032', '02033', '02034', '02035',
       '02036', '02037', '02038', '02039', '02040', '02099'], dtype=object)

In [202]:
df = df[df['totalvotes'] != 0]

In [203]:
df_ak = df[df['state'] == 'ALASKA']
df_rest = df[df['state'] != 'ALASKA']

df_ak_grouped = (
    df_ak.groupby(['year', 'state', 'state_po', 'office', 'candidate', 'party'], as_index=False)
    .agg({'candidatevotes': 'sum', 'totalvotes': 'sum'})
)

df_ak_grouped['county_name'] = 'ALASKA'
df_ak_grouped['county_fips'] = '02000'
df_ak_grouped['version'] = df_ak['version'].mode()[0]
df_ak_grouped['mode'] = df_ak['mode'].mode()[0]

df = pd.concat([df_rest, df_ak_grouped[df.columns]], ignore_index=True)

In [204]:
df.head()

,year,state,state_po,county_name,county_fips,office,candidate,party,candidatevotes,totalvotes,version,mode
0,2000,ALABAMA,AL,AUTAUGA,01001,US PRESIDENT,AL GORE,DEMOCRAT,4942,17208,20220315,TOTAL
1,2000,ALABAMA,AL,AUTAUGA,01001,US PRESIDENT,GEORGE W. BUSH,REPUBLICAN,11993,17208,20220315,TOTAL
2,2000,ALABAMA,AL,AUTAUGA,01001,US PRESIDENT,RALPH NADER,GREEN,160,17208,20220315,TOTAL
3,2000,ALABAMA,AL,AUTAUGA,01001,US PRESIDENT,OTHER,OTHER,113,17208,20220315,TOTAL
4,2000,ALABAMA,AL,BALDWIN,01003,US PRESIDENT,AL GORE,DEMOCRAT,13997,56480,20220315,TOTAL


In [205]:
df_pop = pd.read_csv("data/Population Estimates - US, States, Counties.csv", dtype={
    'state_fips': str,
    'county_fips': str
})

In [206]:
df_pop.head()

,IBRC_Geo_ID,Statefips,Countyfips,Description,Year,Population,Count or Estimate,State or County Release
0,0,0,0,U.S.,1970,203302031,Count,State
1,0,0,0,U.S.,1970,203302037,Estimate,County
2,0,0,0,U.S.,1980,226545805,Count,State
3,0,0,0,U.S.,1980,226542250,Estimate,County
4,0,0,0,U.S.,1990,248790925,Count,State


In [207]:
df_pop.dtypes

IBRC_Geo_ID                 int64
Statefips                   int64
Countyfips                  int64
Description                object
Year                        int64
Population                  int64
Count or Estimate          object
State or County Release    object
dtype: object

In [208]:
df_pop.isna().sum()

IBRC_Geo_ID                0
Statefips                  0
Countyfips                 0
Description                0
Year                       0
Population                 0
Count or Estimate          0
State or County Release    0
dtype: int64

In [209]:
df_pop['Statefips'] = df_pop['Statefips'].astype(str).apply(lambda x: x.zfill(2) if len(x) != 2 else x)
df_pop['Countyfips'] = df_pop['Countyfips'].astype(str).apply(lambda x: x.zfill(3) if len(x) != 3 else x)

df_pop['county_fips'] = df_pop['Statefips'] + df_pop['Countyfips']
df_pop.rename(columns= {'Year':'year'}, inplace=True)


In [210]:
df_pop = df_pop.sort_values(by=['year', 'county_fips', 'Count or Estimate'], ascending=[True, True, True])
df_pop = df_pop.drop_duplicates(subset=['year', 'county_fips'], keep='first')

In [211]:
df_pop.head()

,IBRC_Geo_ID,Statefips,Countyfips,Description,year,Population,Count or Estimate,State or County Release,county_fips
0,0,00,000,U.S.,1970,203302031,Count,State,00000
30,1000,01,000,Alabama,1970,3444354,Count,State,01000
60,1001,01,001,"Autauga County, AL",1970,24460,Estimate,County,01001
88,1003,01,003,"Baldwin County, AL",1970,59382,Estimate,County,01003
116,1005,01,005,"Barbour County, AL",1970,22543,Estimate,County,01005


In [212]:
df_pop.isna().sum()

IBRC_Geo_ID                0
Statefips                  0
Countyfips                 0
Description                0
year                       0
Population                 0
Count or Estimate          0
State or County Release    0
county_fips                0
dtype: int64

In [213]:
missing_fips = set(df['county_fips']) - set(df_pop['county_fips'])
print(sorted(missing_fips))


['2938000']


In [214]:
df_merged = pd.merge(df, df_pop[['year', 'county_fips', 'Population', ]], on=['year', 'county_fips'], how='left')

In [215]:
df_merged.head()

,year,state,state_po,county_name,county_fips,office,candidate,party,candidatevotes,totalvotes,version,mode,Population
0,2000,ALABAMA,AL,AUTAUGA,01001,US PRESIDENT,AL GORE,DEMOCRAT,4942,17208,20220315,TOTAL,43671.0
1,2000,ALABAMA,AL,AUTAUGA,01001,US PRESIDENT,GEORGE W. BUSH,REPUBLICAN,11993,17208,20220315,TOTAL,43671.0
2,2000,ALABAMA,AL,AUTAUGA,01001,US PRESIDENT,RALPH NADER,GREEN,160,17208,20220315,TOTAL,43671.0
3,2000,ALABAMA,AL,AUTAUGA,01001,US PRESIDENT,OTHER,OTHER,113,17208,20220315,TOTAL,43671.0
4,2000,ALABAMA,AL,BALDWIN,01003,US PRESIDENT,AL GORE,DEMOCRAT,13997,56480,20220315,TOTAL,140415.0


In [216]:
df_merged = df_merged.drop(['mode', 'version', 'office', 'candidate', 'state_po'], axis=1)


In [217]:
df_merged.head()

,year,state,county_name,county_fips,party,candidatevotes,totalvotes,Population
0,2000,ALABAMA,AUTAUGA,01001,DEMOCRAT,4942,17208,43671.0
1,2000,ALABAMA,AUTAUGA,01001,REPUBLICAN,11993,17208,43671.0
2,2000,ALABAMA,AUTAUGA,01001,GREEN,160,17208,43671.0
3,2000,ALABAMA,AUTAUGA,01001,OTHER,113,17208,43671.0
4,2000,ALABAMA,BALDWIN,01003,DEMOCRAT,13997,56480,140415.0


In [218]:
df[df['state']=='ALASKA'].head(50)

,year,state,state_po,county_name,county_fips,office,candidate,party,candidatevotes,totalvotes,version,mode
71697,2000,ALASKA,AK,ALASKA,02000,US PRESIDENT,AL GORE,DEMOCRAT,79004,285530,20220315,TOTAL
71698,2000,ALASKA,AK,ALASKA,02000,US PRESIDENT,GEORGE W. BUSH,REPUBLICAN,167398,285530,20220315,TOTAL
71699,2000,ALASKA,AK,ALASKA,02000,US PRESIDENT,OTHER,OTHER,10381,285530,20220315,TOTAL
71700,2000,ALASKA,AK,ALASKA,02000,US PRESIDENT,RALPH NADER,GREEN,28747,285530,20220315,TOTAL
71701,2004,ALASKA,AK,ALASKA,02000,US PRESIDENT,GEORGE W. BUSH,REPUBLICAN,292267,488693,20220315,TOTAL
71702,2004,ALASKA,AK,ALASKA,02000,US PRESIDENT,JOHN KERRY,DEMOCRAT,180365,488693,20220315,TOTAL
71703,2004,ALASKA,AK,ALASKA,02000,US PRESIDENT,OTHER,OTHER,16061,488693,20220315,TOTAL
71704,2008,ALASKA,AK,ALASKA,02000,US PRESIDENT,BARACK OBAMA,DEMOCRAT,123594,326197,20220315,TOTAL
71705,2008,ALASKA,AK,ALASKA,02000,US PRESIDENT,JOHN MCCAIN,REPUBLICAN,193841,326197,20220315,TOTAL
71706,2008,ALASKA,AK,ALASKA,02000,US PRESIDENT,OTHER,OTHER,8762,326197,20220315,TOTAL


In [219]:
df_merged = df_merged.sort_values(by=['year', 'county_fips', 'candidatevotes'], ascending=[True, True, False])

In [220]:
df_merged.head()

,year,state,county_name,county_fips,party,candidatevotes,totalvotes,Population
1,2000,ALABAMA,AUTAUGA,01001,REPUBLICAN,11993,17208,43671.0
0,2000,ALABAMA,AUTAUGA,01001,DEMOCRAT,4942,17208,43671.0
2,2000,ALABAMA,AUTAUGA,01001,GREEN,160,17208,43671.0
3,2000,ALABAMA,AUTAUGA,01001,OTHER,113,17208,43671.0
5,2000,ALABAMA,BALDWIN,01003,REPUBLICAN,40872,56480,140415.0


In [221]:
df_merged = df_merged.drop_duplicates(subset=['year', 'county_fips'], keep='first')

In [222]:
df_merged.head()

,year,state,county_name,county_fips,party,candidatevotes,totalvotes,Population
1,2000,ALABAMA,AUTAUGA,01001,REPUBLICAN,11993,17208,43671.0
5,2000,ALABAMA,BALDWIN,01003,REPUBLICAN,40872,56480,140415.0
8,2000,ALABAMA,BARBOUR,01005,DEMOCRAT,5188,10395,29038.0
13,2000,ALABAMA,BIBB,01007,REPUBLICAN,4273,7101,20826.0
17,2000,ALABAMA,BLOUNT,01009,REPUBLICAN,12667,17973,51024.0


In [223]:
df_merged.isna().sum()

year              0
state             0
county_name       0
county_fips       0
party             0
candidatevotes    0
totalvotes        0
Population        7
dtype: int64

In [224]:
df_merged[df_merged['Population'].isna()]

,year,state,county_name,county_fips,party,candidatevotes,totalvotes,Population
17842,2004,MISSOURI,KANSAS CITY,2938000,DEMOCRAT,104625,141423,NaN
27184,2008,MISSOURI,KANSAS CITY,2938000,DEMOCRAT,120102,153219,NaN
36526,2012,MISSOURI,KANSAS CITY,2938000,DEMOCRAT,105670,136802,NaN
38296,2012,SOUTH DAKOTA,SHANNON,46113,DEMOCRAT,2937,3145,NaN
39794,2012,VIRGINIA,BEDFORD,51515,REPUBLICAN,1527,2805,NaN
45868,2016,MISSOURI,KANSAS CITY,2938000,DEMOCRAT,97735,128601,NaN
59771,2020,MISSOURI,KANSAS CITY,2938000,DEMOCRAT,107660,136645,NaN


In [225]:
df_merged = df_merged.dropna()

In [226]:
df_merged.isna().sum()

year              0
state             0
county_name       0
county_fips       0
party             0
candidatevotes    0
totalvotes        0
Population        0
dtype: int64

In [227]:
df_merged['Population'] = df_merged['Population'].astype('int')

In [228]:
df_merged = df_merged.sort_values(by=['county_fips', 'year'])
df_merged['prev_party'] = df_merged.groupby('county_fips')['party'].shift(1)
df_merged['flipped'] = (df_merged['party'] != df_merged['prev_party']).astype(int)

In [229]:
df_merged.head()

,year,state,county_name,county_fips,party,candidatevotes,totalvotes,Population,prev_party,flipped
1,2000,ALABAMA,AUTAUGA,01001,REPUBLICAN,11993,17208,43671,NaN,1
12449,2004,ALABAMA,AUTAUGA,01001,REPUBLICAN,15196,20081,48366,REPUBLICAN,0
21791,2008,ALABAMA,AUTAUGA,01001,REPUBLICAN,17403,23641,53277,REPUBLICAN,0
31133,2012,ALABAMA,AUTAUGA,01001,REPUBLICAN,17379,23932,54970,REPUBLICAN,0
40475,2016,ALABAMA,AUTAUGA,01001,REPUBLICAN,18172,24973,55302,REPUBLICAN,0


In [230]:
df_merged.to_csv('data/cleaned_data.csv', index=False)